<a href="https://colab.research.google.com/github/Nitesh-K-Singh/blogs/blob/main/topic_wise/credit_risk/Credit_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Libraries

In [ ]:
import pandas as pd
import numpy as np

#Introduction (Done for now)

Here we will solve a credit risk problem end to end. We will take a datset that will have entries at the individual level. More specifically, each row in the dataset will correspond to a customer who in the past has availed a loan. We will have data on attribuites of the individual at the time of loan, the purpose of the loan, and the interest rate and amount of loan granted. This will form our feature set. We will also have a 0-1 label corresponding to each individual with 1 representing that particular indiviudla defaulted on their loan.

- The first task will be to do opportunity sizing of the problem. How big is the defaulting  propblem? How to control data leakage here?
- If it is a big problem, then next logical task is what to do about it. Since when a consumer comes for a loan we will only observe their features and not their label (which will only become observable in the future if we givce them loan). We have to make a decision to give or not give a loan based on these features alone. If we give loan to people who would in future default, then we will lose money. If we do not give loan to people who would not have defaulted we will forego money we could have made from the interest.  So our solution will have to balance these constraints.
- We will build an ML model (without data leakage) on this datset and do an evaluation of how much money we could have saved had our model been used to give/not give loans.
- We will assume that future consumers will be drawn from the same distribution  as this dataset and therefore out of sample accuracy/errors of this dataset is a valid proxy for accuracy/errors on our future consumers. With this very important assumption we will make a case for using our developed model for credit decisions.



***To Do***: he first task will be to do opportunity sizing of the problem. How big is the defaulting  propblem? How to control data leakage here?

#Distinctions (Done for now)

How this notebook differs from other Medium/Kaggle Notebooks?


Correcting some important mistakes:

1.  Data Leakage: Don't preprocess the data first. Do data splitting first. [Scikit learn](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage-during-pre-processing) , [Abu Mustafa](https://www.google.com/url?q=https%3A%2F%2Fwork.caltech.edu%2Flibrary%2F173.html)
2.  Interactions Matter: Just because a feature X is balanced across defaualters and non-defaulters it does not imply that X does not matter. It can still interact with other features and have an effect in subgrouos.
3. Calibration Matters: Do a Train-Validate-Calibrate-Test split.
4. Embed it into a business framework
  *   Opportunity Sizing in beginning
  *   Expected Payoff from using the model at end

5. There are a lot of values in the city. So one hot encoding would not be a good choice for categorical variable processing. Label encoding is a better choice
6.

***To do:*** https://stats.stackexchange.com/questions/95797/how-to-split-the-dataset-for-cross-validation-learning-curve-and-final-evaluat?rq=1

***To do***: https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio

#Table of Contents


Steps: Change it to correct for mistakes above.


3. Knowing the Data
4. Exploring some variables
5. Correlation of the Data
6. Preprocessing
7. Traing the moidels: model1,2,3,4,5,..
8. Evaluation and Fine-Tuning: Score Values, Cross Validation, ROC curves
9.





Data Used: https://www.kaggle.com/datasets/laotse/credit-risk-dataset?source=post_page-----481b3a398e0--------------------------------

Hosted at:

#Data (Done for now)

1. Read the data.
2. Check missing values.
3. See description and data type of columns.
4. Be very careful to avoid data leakage. Don't do any kind of EDA or pre-processing here.

See this awesome segment for data leakage/snooping: [Abu Mustafa](https://work.caltech.edu/library/173.html)

***To Do***: Think about what else we can do in this section while still avoiding data leakage.



In [ ]:
path = 'https://raw.githubusercontent.com//Nitesh-K-Singh/blogs/blame/0f1e49d066de8de6726814111f79a19dcb657e21/topic_wise/credit_risk/data/credit_risk_dataset.csv'

# path = 'https://raw.githubusercontent.com/Nitesh-K-Singh/blogs/blame/0f1e49d066de8de6726814111f79a19dcb657e21/topic_wise/credit_risk/data/credit_risk_dataset.csv'
df = pd.read_csv(path)
df.head(1)

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Can use df.dtypes but trying to use train set from here on to avoid accidental data leakege

print(df.dtypes)

person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
dtype: object



The dataframe contains historical data at an individual level data. So each row represents an individual, their attributes( income, home ownership, years of experience, credit history- tenure and default) at the time when loan was sanctioned; puprose, interest rate, and amount of the sanctioned loan; and a binary indicator representing whether they defaulted or not.

| Feature                  | Data Type | Description                       |
| -------------------------| --------- | --------------------------------- |
| person_age | continuous | age of the person |
| person_income | continuous | annual income of the person |
| person_home_ownership| categorical | Rent/Mortrage/Own/Other|
| person_emp_length | continuous | Employment length (in years) |
| cb_person_default_on_file| categorical | Historical default- yes/no |
| cb_person_cred_hist_length| continuous | Credit history length in years |
| loan_intent | categorical | purpose for loan |
| loan_int_rate  | continuous | interest rate of the sanctioned loan |
| loan_amnt | continuous | amount of sanctioned loan |
| loan_percent_income | continuous | (loan amt/income)*100 |
| loan_grade | categorical | A/B/C/D/E/F/G|
| loan_status| 0-1 | 1 if default , 0 otherwise (target variable) |



In [ ]:
# Suggests missing data in the person_emp_length and loan_int_rate columns

df.count()

,0
person_age,32581
person_income,32581
person_home_ownership,32581
person_emp_length,31686
loan_intent,32581
loan_grade,32581
loan_amnt,32581
loan_int_rate,29465
loan_status,32581
loan_percent_income,32581


In [ ]:
# Confirming Missinhg Values. True means there is atleast one missing value.
print(df.isnull().values.any())

# df.isnull() is a dataframe of same shape as df with entries  False/True depending on missing values

True


In [ ]:
# Number of missing values in each column, Total number of missing values
print(df.isnull().sum(), df.isnull().sum().sum())

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64 4011


In [ ]:
# number of unique values in each column

print(df.nunique(axis=0))

person_age                      58
person_income                 4295
person_home_ownership            4
person_emp_length               36
loan_intent                      6
loan_grade                       7
loan_amnt                      753
loan_int_rate                  348
loan_status                      2
loan_percent_income             77
cb_person_default_on_file        2
cb_person_cred_hist_length      29
dtype: int64


#Test-Calibrate-Validate-Train Split (Done for now)

We will reshuffle the dataframe first. On why this is necessary see this link - [reshuffle.](https://https://stats.stackexchange.com/questions/550399/what-is-the-role-of-shuffle-in-train-test-split
)
Once that is done, we will divide the dataset into 4 parts:
1. test set: approx 20% of the dataset. We will make sure to not look at this data. Once everything is done and ***one final*** model selected (after validation, calibration, etc) we will use this dataset (programatically) to get an estimate of expected accuracy/error of our model.
2. calibration set: approx 1000 data points to calibrate the winning model after validation
3. validation set: approx 20% of the remaining data. This is used for hyperparameter tuning and model selection.
4. train set: remaining data (approx 60%). This is the data we will have access to for doing EDA and model training.


There is a randomness in these splits (controlled by reshguffling of the dataframe). So each different starting point (test, calibrate, validate, train) tuple can  lead to a different winning model.

To take care of this we can iterate over these splits repeatedly (say 100 times) and see if the same algorithm class (could be with different learned parameters and tuned hyperparameters) wins.

***To Do:*** Is there another way to take care of this?

In [ ]:

# Shuffle the dataset:
# https://stats.stackexchange.com/questions/550399/what-is-the-role-of-shuffle-in-train-test-split


# Source of randomness

df = df.sample(frac=1, random_state=42)


# Number of data points in each type
n = len(df)
n_test = int(0.2*n)
n_calibrate = int(1000)
n_validate = int((n - n_test - n_calibrate)*0.2)
n_train = n - n_test - n_calibrate - n_validate

# making different dfs.
# we will work only with train,
# use validate only for model selection.
# use calibrate on the final selected model
# use test only at last, and that too programatically

test, calibrate, validate, train = np.split(df, [n_test, n_test + n_calibrate,  n_test + n_calibrate + n_validate])



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Run the following command and see differences in the rows to check yourself that shuffling is working



```
Code Block 1:

for i in range(10):
  data = df
  data = data.sample(frac=1, random_state= i)
  print(data.index[:10].to_list())
```

This shuffles the dataframe each time with a different seed. Then it returns
a list of index of first 10 rows according to original dataframe. On each iteration the list will change.



In order to see how setting seed is important for replicability, run the following code

```
Code Block 2:

for i in range(10):
  data = df
  data = data.sample(frac=1, random_state= 7)
  print(data.index[:10].to_list())
```
On each iteration we will get the same list.



Note that the following code will give different results on each iteration despite fixing the seed:



```
Code Block 3:

for i in range(10):
  df = df.sample(frac=1, random_state= 7)
  print(df.index[:10].to_list())
```

This is because each time we are updating df itself...





In [ ]:
#######################
print('Code Block 1')

for i in range(10):
  data = df
  data = data.sample(frac=1, random_state= i)
  print(data.index[:10].to_list())

print('Verify if above results are different')

#######################
print('Code Block 2')

for i in range(10):
  data = df
  data = data.sample(frac=1, random_state= 7)
  print(data.index[:10].to_list())

print('Verify if above results are same')



#######################
print('Code Block 3')

for i in range(10):
  df = df.sample(frac=1, random_state= 7)
  print(df.index[:10].to_list())

for i in range(10):
  df = df.sample(frac=1, random_state= 7)
  print(df.index[:10].to_list())

print('Think about why above results are different')



Code Block 1
[16982, 26975, 26570, 12284, 3417, 2047, 27948, 23474, 8273, 2699]
[27284, 220, 1541, 12256, 14177, 9881, 31537, 24396, 7331, 31069]
[14873, 4776, 6157, 19267, 21749, 17395, 5991, 16159, 12999, 23994]
[4592, 6268, 18368, 26284, 11427, 25317, 25163, 4597, 18680, 15096]
[23537, 29819, 4323, 619, 7619, 1804, 25356, 15995, 25724, 26676]
[17968, 27165, 6921, 9332, 3544, 17200, 8850, 15945, 9018, 22802]
[4043, 16741, 9039, 21976, 26782, 5751, 3355, 27546, 11064, 24520]
[21062, 3510, 11480, 10048, 17179, 3343, 2603, 14778, 17822, 30951]
[7239, 30302, 8011, 10148, 17776, 5817, 1651, 19347, 1585, 27194]
[24263, 8132, 19094, 5561, 15875, 25239, 23281, 23001, 12508, 4757]
Verify if above results are different
Code Block 2
[21062, 3510, 11480, 10048, 17179, 3343, 2603, 14778, 17822, 30951]
[21062, 3510, 11480, 10048, 17179, 3343, 2603, 14778, 17822, 30951]
[21062, 3510, 11480, 10048, 17179, 3343, 2603, 14778, 17822, 30951]
[21062, 3510, 11480, 10048, 17179, 3343, 2603, 14778, 17822, 3

There is no randomness in np.split().
For the list of numbers [a, b, c]



```
np.split(df, [a,b,c])
```

will return four dataframes

```
df[:a], df[a:b], df[b:c], df[c:]
```


  
  

Once df is nailed down (index frozen in above command) there is no randomness left. Can run the below command and verify that output is same



```
n = len(df)
n_test = int(0.2*n)
n_calibrate = int(1000)
n_validate = int((n - n_test - n_calibrate)*0.2)
n_train = n - n_test - n_calibrate - n_validate

for i in range(10):
  np.random.seed(i)
  test, calibrate, validate, train =   np.split(df, [n_test, n_test + n_calibrate,  n_test + n_calibrate + n_validate])
  print(train.index[:10].to_list())
  
```




In [ ]:
# No randomness here

n = len(df)
n_test = int(0.2*n)
n_calibrate = int(1000)
n_validate = int((n - n_test - n_calibrate)*0.2)
n_train = n - n_test - n_calibrate - n_validate

for i in range(10):
  np.random.seed(i)
  test, calibrate, validate, train =   np.split(df, [n_test, n_test + n_calibrate,  n_test + n_calibrate + n_validate])
  print(train.index[:10].to_list())


[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]
[8586, 8047, 5866, 23055, 19297, 14990, 21202, 26485, 17740, 31076]


#Train Set

Here we will

1. Do Basic EDA of data (correlation plots, ?)
2. Impute Missing Values.
3. Encode categorical features.
4. Take care of any outliers.
4. Make any features we want.
5. Look at correlational numbers, etc.
6. Feature Scaling?
Any data pre-processing we do here has to be replicated for validation, calibration, and test set. Make a pipeline and reuse it so that there is no discrepancy due to this.

##Preparing Data (Done for now)


The dataframe contains historical data at an individual level data. So each row represents an individual, their attributes( income, home ownership, years of experience, credit history- tenure and default) at the time when loan was sanctioned; puprose, interest rate, and amount of the sanctioned loan; and a binary indicator representing whether they defaulted or not.

| Feature                  | Data Type | Description                       |
| -------------------------| --------- | --------------------------------- |
| person_age | continuous | age of the person |
| person_income | continuous | annual income of the person |
| person_home_ownership| categorical | Rent/Mortrage/Own/Other|
| person_emp_length | continuous | Employment length (in years) |
| cb_person_default_on_file| categorical | Historical default- yes/no |
| cb_person_cred_hist_length| continuous | Credit history length in years |
| loan_intent | categorical | purpose for loan |
| loan_int_rate  | continuous | interest rate of the sanctioned loan |
| loan_amnt | continuous | amount of sanctioned loan |
| loan_percent_income | continuous | (loan amt/income)*100 |
| loan_grade | categorical | A/B/C/D/E/F/G|
| loan_status| 0-1 | 1 if default , 0 otherwise (target variable) |



###Missing Values (Done for now)

In [ ]:
# Missing values

print(train.isnull().sum())
print(train.isnull().sum().sum())
print(len(train))

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              552
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 1913
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64
2465
20052


There are 2446 rows (at the most) where data is missing. Since even after dropping these we still have more than 17000 data points, we are going to drop the missing values.

Dropping missing values has two implications:
1. Data for training reduces.
2. We are assuming that data is missing at random (MAR). If there is selection bias in the missing data then this will create problem.

Point 1 above is easy to deal with, we can pick any method from many imputations methods available: https://scikit-learn.org/stable/modules/impute.html
  
Point 2 however is very difficult to justify.

In [ ]:
train_dropped = train.dropna()
print(train_dropped.count())

person_age                    17631
person_income                 17631
person_home_ownership         17631
person_emp_length             17631
loan_intent                   17631
loan_grade                    17631
loan_amnt                     17631
loan_int_rate                 17631
loan_status                   17631
loan_percent_income           17631
cb_person_default_on_file     17631
cb_person_cred_hist_length    17631
dtype: int64


###Encoding (Done for now)

In [ ]:
for x in train_dropped.columns:
  if train_dropped[x].dtype == 'object':
    value = set(train_dropped[x].values)
    print(value, len(value))


{'RENT', 'MORTGAGE', 'OTHER', 'OWN'} 4
{'EDUCATION', 'DEBTCONSOLIDATION', 'VENTURE', 'PERSONAL', 'MEDICAL', 'HOMEIMPROVEMENT'} 6
{'D', 'C', 'F', 'B', 'A', 'E', 'G'} 7
{'Y', 'N'} 2


Since the cardinality of categorical features is small we can do one hot encoding here. For other approaches including target encoding see https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features



In [ ]:
# dtype int to get 1,0 instead of true/false.
# drop_first = True: Whether to get k-1 dummies out of k categorical levels
# by removing the first level. Avoids Multicollinearity.

train_dropped_encoded = pd.get_dummies(train_dropped, dtype = int, drop_first=True )
train_dropped_encoded.head(5)



,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
8586,24,22100,4.0,8000,13.23,1,0.36,2,0,0,...,0,0,0,0,1,0,0,0,0,0
8047,24,45000,3.0,7200,10.65,0,0.16,4,0,0,...,0,0,0,1,0,0,0,0,0,0
5866,24,47000,4.0,6000,7.66,0,0.13,3,0,0,...,0,0,0,0,0,0,0,0,0,0
19297,28,46000,0.0,2500,6.91,0,0.05,8,0,0,...,1,0,0,0,0,0,0,0,0,0
21202,29,45600,7.0,9600,10.36,0,0.21,5,0,0,...,0,0,1,1,0,0,0,0,0,0


In [ ]:
# makes sense, we get 11 new columns
print(len(train_dropped.columns), len(train_dropped_encoded.columns))

12 23


###Outliers (Done for now)

In [ ]:
# df.describe() #Not allowed - Data Leakage

train_dropped_encoded.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,17631.0,27.700811,6.290720,20.00,23.00,26.00,30.00,144.00
person_income,17631.0,66222.075605,51317.646420,4080.00,39000.00,55200.00,80000.00,2039784.00
person_emp_length,17631.0,4.770518,4.141550,0.00,2.00,4.00,7.00,123.00
loan_amnt,17631.0,9615.648574,6256.865197,500.00,5000.00,8000.00,12250.00,35000.00
loan_int_rate,17631.0,11.044914,3.232890,5.42,7.90,11.03,13.48,23.22
loan_status,17631.0,0.214735,0.410650,0.00,0.00,0.00,0.00,1.00
loan_percent_income,17631.0,0.169060,0.105616,0.00,0.09,0.15,0.23,0.78
cb_person_cred_hist_length,17631.0,5.774828,4.024923,2.00,3.00,4.00,8.00,30.00
person_home_ownership_OTHER,17631.0,0.003460,0.058720,0.00,0.00,0.00,0.00,1.00
person_home_ownership_OWN,17631.0,0.075265,0.263826,0.00,0.00,0.00,0.00,1.00


We can see outliers in two features:

1. person_age = 144: This is not possible and most likely an error. There are three data points in the training set that have age> 100. As a sanity check I am going to delete any row that has age>= 100.  



```
train_dropped_encoded[ train_dropped_encoded['person_age'] > 100 ].T

```



2. loan_percent_income	=	0.00 : there are 4 data points in training set where loan_percent_income is 0. This is because any number < 0.1 seems to be rounded off to 0 in the dataset. So I am leaving these untouched.''



```
print(train_dropped_encoded[ train_dropped_encoded['loan_percent_income'] == 0].T)
print(6400.0/	1440000.0)
```



Other options in case 1 would have been to cap or impute the values of these outliers.
In case 2 it would have been to replace 0 with actual values even if <0.1.

Reference at your own risk: https://syedabis98.medium.com/dealing-with-outliers-in-data-science-techniques-and-best-practices-a08172643b7a

In [ ]:
train_dropped_encoded[ train_dropped_encoded['person_age'] > 100 ].T

,575,81,183
person_age,123.00,144.00,144.00
person_income,80004.00,250000.00,200000.00
person_emp_length,2.00,4.00,4.00
loan_amnt,20400.00,4800.00,6000.00
loan_int_rate,10.25,13.57,11.86
loan_status,0.00,0.00,0.00
loan_percent_income,0.25,0.02,0.03
cb_person_cred_hist_length,3.00,3.00,2.00
person_home_ownership_OTHER,0.00,0.00,0.00
person_home_ownership_OWN,0.00,0.00,0.00


In [ ]:
print(train_dropped_encoded[ train_dropped_encoded['loan_percent_income'] == 0].T)
print(6400.0/	1440000.0)

                                 27877       30049      31916
person_age                       30.00       42.00      43.00
person_income                522000.00  2039784.00  780000.00
person_emp_length                11.00        0.00       2.00
loan_amnt                      2500.00     8450.00    1000.00
loan_int_rate                     7.43       12.29       8.94
loan_status                       0.00        0.00       0.00
loan_percent_income               0.00        0.00       0.00
cb_person_cred_hist_length        9.00       15.00      11.00
person_home_ownership_OTHER       0.00        0.00       0.00
person_home_ownership_OWN         0.00        0.00       0.00
person_home_ownership_RENT        0.00        1.00       0.00
loan_intent_EDUCATION             0.00        0.00       0.00
loan_intent_HOMEIMPROVEMENT       1.00        0.00       1.00
loan_intent_MEDICAL               0.00        0.00       0.00
loan_intent_PERSONAL              0.00        0.00       0.00
loan_int

In [ ]:
# Final Training Set
outlier = train_dropped_encoded[ train_dropped_encoded['person_age'] > 100 ].index
train_set = train_dropped_encoded.drop(outlier)

print(len(train_set), len(outlier), len(train_dropped_encoded))

17628 3 17631


Putting all the pre-processing step together since the same will be used for validation, calibration, and test sets.

1. Dropping Null Values


```
train_dropped = train.dropna()
print(train_dropped.count())

```
2. One hot encoding



```
train_dropped_encoded = pd.get_dummies(train_dropped, dtype = int, drop_first=True )
train_dropped_encoded.head(5)

```



3. Outlier Removal



```
outlier = train_dropped_encoded[ train_dropped_encoded['person_age'] > 100 ].index
train_set = train_dropped_encoded.drop(outlier)

print(len(train_set), len(outlier), len(train_dropped_encoded))
```

All these three steps can be combined together as shown below. The function prepare_data returns exactly same dataframe. So when working with validation, calibtration, test data we will directly call this function. Note thaty we are not supposed to look at test, calibrate, validate data to build hypothesis; and therefore we zeroed in on pre-processing or feature engineering steps on training data only.

In [ ]:
# Combining all pre-processing steps into one function

def prepare_data(df):
  # Dropping Null Values
  df_dropped = df.dropna()

  # One hot encoding
  df_dropped_encoded = pd.get_dummies(df_dropped, dtype = int, drop_first=True )

  # Outlier Removal
  outlier = df_dropped_encoded[ df_dropped_encoded['person_age'] > 100 ].index
  df_set = df_dropped_encoded.drop(outlier)
  return df_set


# Pre-procesing the train data using one function
a = prepare_data(train)

# Checking if data obtained from this function is same as the one we obtained before
a.equals(train_set)

True

In [ ]:
# Preparing Validation, Calibration, and Test Data.

validation_set = prepare_data(validate)
calibration_set = prepare_data(calibrate)
test_set = prepare_data(test)

##Feature Scaling

1. https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html
To do or not? If features not predictive it will add noise but if predictive better.
2. https://stats.stackexchange.com/questions/611349/is-there-any-downside-to-scaling-a-dataset

If you scale on this dataset, how do you make inference on a new data?

"where a decision tree would have a decision node on "age <45 years-old" (which is understandable), with scaled data you might end up with that node reading "age <0.021 years-old" (which is meaningless)."

##EDA

### Descriptive Stats


| Feature                  | Data Type | Description                       |
| -------------------------| --------- | --------------------------------- |
| person_age | continuous | age of the person |
| person_income | continuous | annual income of the person |
| person_home_ownership| categorical | Rent/Mortrage/Own/Other|
| person_emp_length | continuous | Employment length (in years) |
| cb_person_default_on_file| categorical | Historical default- yes/no |
| cb_person_cred_hist_length| continuous | Credit history length in years |
| loan_intent | categorical | purpose for loan |
| loan_int_rate  | continuous | interest rate of the sanctioned loan |
| loan_amnt | continuous | amount of sanctioned loan |
| loan_percent_income | continuous | (loan amt/income)*100 |
| loan_grade | categorical | A/B/C/D/E/F/G|
| loan_status| 0-1 | 1 if default , 0 otherwise (target variable) |

In [ ]:
train_set.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
count,17628.000000,1.762800e+04,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,...,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000,17628.000000
mean,27.682210,6.620328e+04,4.770762,9615.515090,11.044770,0.214772,0.169071,5.775357,0.003460,0.075278,...,0.186295,0.171375,0.175006,0.320343,0.199002,0.113172,0.030633,0.006751,0.001815,0.177388
std,6.126148,5.129334e+04,4.141842,6256.705975,3.233098,0.410676,0.105612,4.025056,0.058725,0.263847,...,0.389355,0.376847,0.379983,0.466621,0.399261,0.316812,0.172326,0.081887,0.042569,0.382008
min,20.000000,4.080000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,3.900000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,5.520000e+04,4.000000,8000.000000,11.030000,0.000000,0.150000,4.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,30.000000,8.000000e+04,7.000000,12250.000000,13.480000,0.000000,0.230000,8.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,80.000000,2.039784e+06,123.000000,35000.000000,23.220000,1.000000,0.780000,30.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_set.corr()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
person_age,1.000000,0.142618,0.169624,0.055562,0.015704,-0.019412,-0.037176,0.877543,-0.009716,0.010730,...,0.023656,0.026983,-0.003695,-0.008721,0.003104,0.022975,-0.007222,0.003146,0.011784,0.008381
person_income,0.142618,1.000000,0.157652,0.320674,-0.008172,-0.166980,-0.296371,0.121817,0.002813,-0.029745,...,-0.038810,0.003225,0.009154,0.001759,-0.015442,-0.018922,0.020958,0.012420,0.012070,0.003245
person_emp_length,0.169624,0.157652,1.000000,0.111891,-0.058764,-0.083952,-0.056217,0.145974,-0.024727,0.034584,...,-0.011404,0.014667,0.015976,-0.015455,-0.040887,-0.003574,-0.018934,-0.013502,0.015231,-0.023133
loan_amnt,0.055562,0.320674,0.111891,1.000000,0.135071,0.108394,0.579142,0.044660,0.012360,-0.014016,...,-0.033008,0.002694,0.003250,0.044660,-0.037790,0.072487,0.087407,0.072487,0.055967,0.039528
loan_int_rate,0.015704,-0.008172,-0.058764,0.135071,1.000000,0.338976,0.115174,0.020394,0.016404,-0.011519,...,0.002758,-0.005817,-0.006373,-0.006118,0.373938,0.477920,0.327644,0.195659,0.126191,0.498540
loan_status,-0.019412,-0.166980,-0.083952,0.108394,0.338976,1.000000,0.374227,-0.010622,0.004467,-0.100526,...,0.061269,-0.014966,-0.083096,-0.090832,-0.021943,0.324205,0.186797,0.108712,0.078296,0.174086
loan_percent_income,-0.037176,-0.296371,-0.056217,0.579142,0.115174,0.374227,1.000000,-0.028108,0.012867,0.044000,...,0.010016,0.002160,-0.004559,0.034862,-0.012112,0.073167,0.053463,0.042511,0.030534,0.023277
cb_person_cred_hist_length,0.877543,0.121817,0.145974,0.044660,0.020394,-0.010622,-0.028108,1.000000,-0.008232,0.013414,...,0.021420,0.026355,0.000149,-0.007957,0.007909,0.022118,-0.009299,0.003741,0.012313,0.008466
person_home_ownership_OTHER,-0.009716,0.002813,-0.024727,0.012360,0.016404,0.004467,0.012867,-0.008232,1.000000,-0.016813,...,-0.008346,0.011654,0.010995,-0.007331,-0.000337,0.006393,0.011948,0.006939,-0.002513,0.010569
person_home_ownership_OWN,0.010730,-0.029745,0.034584,-0.014016,-0.011519,-0.100526,0.044000,0.013414,-0.016813,1.000000,...,-0.014476,-0.002519,0.094932,0.000417,-0.009196,0.001236,-0.020775,-0.005141,0.008037,-0.004725


In [ ]:
outcome_corr = train_set.corr().loc['loan_status']


KeyError: True

### Feature Engineering

##Training

#References

All references in one place:

1. Data Leakage/Snooping:
2. Shuffling:
3. Calibration Set:
4. Interaction Effects:
5. Decision Theoretic Framework: Expected Profit/ Loss
6. Missing Value Imputation:
7. Encoding Categorical Features:
8.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Dump

In [ ]:
# for x in list(train_set.columns):
#   y = x.replace("'","")
#   print(y)

#Caveats

1. Impact of dropping missing values:
2.